# Trending Value Analysis: The Best from Growth and Value

By Joshua Genao

---

In this notebook we will analyze the Trending Value Portfolio that is mentioned in the Fourth edition of "What Works On Wall Street" by James O'Shaugnessy. In the "Uniting The Best From Growth and Value" chapter, O'Shaugnessy looks at uniting the best growth and value factors to produce a portfolio that is known as the Trending Value.

We will be looking at the Value Composite Two which is composed of ranking the following factors:
<ol>
    <li>Price-to-book</li>
    <li>Price-to-earnings</li>
    <li>Price-to-sales</li>
    <li>EBITDA/EV</li>
    <li>Price-to-cash flow</li>
    <li>Shareholder yield</li>
</ol>

We assign a percentile ranking (from 1 to 100) for each stock in the All Stock Universe. The book mentions that the All Stock Universe comprises of any stock with a market capitalization above $200 million.

Stocks included in the Trending Value strategy must:
<ol>
    <li>Be a member of the All Stock Universe</li>
    <li>Be in decile 1 of the composited Value Factor Two (10% of the best stocks with best valuation scores across all 6 factors)</li>
    <li>Buy the 25 stocks with the best 6-month price appreciation</li>
</ol>

The portfolio is rebalanced every year.

Note: James O'Shaughnessy is using Compustat data ranging from 1964 to 2009.

## Trending Value Portfolio
Lets first construct the portfolio by ranking all the value factors above from the All Stock Universe

In [ ]:
from quantopian.pipeline import Pipeline
from quantopian.pipeline import Pipeline, CustomFilter
from quantopian.pipeline.factors import CustomFactor
from quantopian.pipeline.data import Fundamentals
from quantopian.pipeline.data.builtin import USEquityPricing
import numpy as np
import pandas as pd

Quantiles class was created in order to change any results that contains a NaN value into a neutral rank of 50

In [ ]:
class Ebitda_to_EV(CustomFactor):
    '''
    The book uses EBITDA/EV. Fundamental data gives us access to EV/EBITDA. 
    EBITDA/EV = 1/(EV/EBITDA)
    If result gives us an infinite number, that is changed to NaN. This will 
    be handled when passing it to the Quantiles function
    '''
    window_length = 1
    inputs=[Fundamentals.ev_to_ebitda]
    
    def compute(self, today, assets, out, ev_ebitda):
        result = 1 / ev_ebitda[-1]
        result[np.isinf(result)] = np.nan
        out[:] = result

In [ ]:
class Momentum(CustomFactor):
    '''
    Momentum is calculated by determining the price appreciation.
    Price appreciation = current price - previous price / previous price
    This will gives us a decimal value.
    '''
    inputs=[USEquityPricing.open]
    def compute(self, today, assets, out, price):
        out[:] = (price[-1] - price[0]) / price[-1]

In [ ]:
class Quantiles(CustomFactor):
    window_length = 1
    params = ('q_type', 'q_value',)
    fill_value = 50 
    
    def compute(self, today, assets, out, factor, q_type, q_value):
        try:
            if q_type == 'quantiles':
                result = pd.qcut(factor, q_value, labels=False) + 1   
            elif q_type == 'bins':
                result = pd.cut(factor, q_value, labels=False) + 1
            else:
                raise ValueError('Either quantiles or bins should be provided')
            
            result[np.isinf(result)] = self.fill_value
            result[np.isnan(result)] = self.fill_value
            out[:] = result  
        except:
            out[:] = np.nan

In [ ]:
def make_pipeline():
    # All Stocks universe consist of stocks with a market capitalization in excess of $200 mil
    all_stock_universe = Fundamentals.market_cap.latest > 200000000
    
    # Get the latest of all factors from the Value Composite 2
    '''
    pb_ratio = Fundamentals.pb_ratio.latest
    pe_ratio = Fundamentals.pe_ratio.latest
    ps_ratio = Fundamentals.ps_ratio.latest
    ebitda_ev = Ebitda_to_EV()
    price_to_cashflow = Fundamentals.pcf_ratio.latest
    shareholder_yield = Fundamentals.total_yield.latest
    '''
    #ebitda_ev = Ebitda_to_EV()

    
    # Factors that receives a rank of 100 in the lowest 1 percent of the universe and
    # a rank of 1 in the highest 1 percent of the universe
    '''
    pb_ratio_rank = pb_ratio.rank(ascending=False, mask=all_stock_universe)
    pe_ratio_rank = pe_ratio.rank(ascending=False, mask=all_stock_universe)
    ps_ratio_rank = ps_ratio.rank(ascending=False, mask=all_stock_universe)
    price_to_cashflow_rank = price_to_cashflow.rank(ascending=False, mask=all_stock_universe)
    '''
    
    # Factors that receives a rank of 100 in the highest 1 percent of the universe and
    # a rank of 1 in the lowest 1 percent of the universe
    cash_return = Fundamentals.cash_return.latest.rank(mask=all_stock_universe)
    fcf_yield = Fundamentals.fcf_yield.latest.rank(mask=all_stock_universe)
    roic = Fundamentals.roic.latest.rank(mask=all_stock_universe)
    ltd_to_eq = Fundamentals.long_term_debt_equity_ratio.latest.rank(mask=all_stock_universe)
    
    #shareholder_yield_rank = shareholder_yield.rank(mask=all_stock_universe)
    #ebitda_ev_rank = ebitda_ev.rank(mask=all_stock_universe)
    
    cash_return_quantiles = Quantiles(inputs=[cash_return], q_type='quantiles', q_value=100)
    fcf_yield_quantiles = Quantiles(inputs=[fcf_yield], q_type='quantiles', q_value=100)
    roic_quantiles = Quantiles(inputs=[roic], q_type='quantiles', q_value=100)
    ltd_to_eq_quantile = Quantiles(inputs=[ltd_to_eq], q_type='quantiles', q_value=100)
    #pb_ratio_quantiles = Quantiles(inputs=[pb_ratio_rank], q_type='quantiles', q_value=100)
    #pe_ratio_quantiles = Quantiles(inputs=[pe_ratio_rank], q_type='quantiles', q_value=100) 
    #ps_ratio_quantiles = Quantiles(inputs=[ps_ratio_rank], q_type='quantiles', q_value=100)
    #ebitda_ev_rank_quantiles = Quantiles(inputs=[ebitda_ev_rank], q_type='quantiles', q_value=100)
    #price_to_cashflow_quantiles = Quantiles(inputs=[price_to_cashflow_rank], q_type='quantiles', q_value=100)
    #shareholder_yield_quantiles = Quantiles(inputs=[shareholder_yield_rank], q_type='quantiles', q_value=100)
    
    
    #score = pb_ratio_quantiles + pe_ratio_quantiles + ps_ratio_quantiles + ebitda_ev_rank_quantiles + price_to_cashflow_quantiles + shareholder_yield_quantiles
    #score_rank = score.rank(ascending=False)
    score = ( cash_return_quantiles + fcf_yield_quantiles + ltd_to_eq_quantile + roic_quantiles ).rank(ascending=False)
    #score_decile = Quantiles(inputs=[score_rank], q_type='quantiles', q_value=10)
    
    top_quality = score.top(100, mask=all_stock_universe)
    #momentum_6mon = Momentum(window_length = 180, mask=all_stock_universe)
    #momentum_rank = momentum_6mon.rank(ascending=False, mask=score_decile.eq(1))
    momentum = Momentum(window_length = 180, mask=top_quality)
    # assign a percentile ranking (from 1 to 100)
    # e.g if a stock has PE ratio that is in the lowest 1 percent it receives a rank of 100
    
    return Pipeline(
        columns={
            'factor' : momentum
        },
        screen=top_quality
    )
    

Lets determine if the data seems correct. The following factors should have a rank of 100 if it is in the lowest 1 percent of the universe:
<ul>
    <li>Price-to-book</li>
    <li>Price-to-earnings</li>
    <li>Price-to-sales</li>
    <li>Price-to-cashflow</li>
</ul>

In [ ]:
# Import run_pipeline method
from quantopian.research import run_pipeline

# Specify a time range to evaluate
period_start = '2010-01-01'
period_end = '2019-01-01'

# Execute pipeline over evaluation period
pipeline_output = run_pipeline(
    make_pipeline(),
    start_date=period_start,
    end_date=period_end
)

print "There are %d assets in this universe." % len(pipeline_output.index.levels[1])

###Value Factor Two
Lets determine how well the Value Factor Two portfolio does

In [ ]:
import alphalens

#Gets each unique stock ticker and puts it in assets
assets = pipeline_output.index.levels[1].unique()

#gets pricing data. Needs a month before and after. Dunno why.
pricing = get_pricing(assets, start_date='2010-01-01', end_date='2019-01-01', fields='open_price')

In [ ]:
# Ingest and format data
factor_data = alphalens.utils.get_clean_factor_and_forward_returns(pipeline_output,
                                                                   pricing,
                                                                   periods=(30,60,90),
                                                                   quantiles=10
                                                                  )

In [ ]:
alphalens.tears.create_information_tear_sheet(factor_data)

In [ ]:
mean_return_by_q, std_err_by_q = alphalens.performance.mean_return_by_quantile(factor_data,
                                                                               by_group=False)

In [ ]:
mean_return_by_q.head()

In [ ]:
alphalens.plotting.plot_quantile_returns_bar?

In [ ]:
alphalens.tears.create_returns_tear_sheet(factor_data);